In [1]:
import pandas as pd
import numpy as np
import os
import pytesseract  # For OCR
from PIL import Image  # For image processing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import random
import re
import cv2  # For image processing
import matplotlib.pyplot as plt  # For visualization
import torch  # For BERT and PyTorch
from transformers import BertTokenizer, BertForTokenClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import requests  # For downloading images

d:\python3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set the path to your dataset
DATASET_PATH = 'D:\Projects\Amazon-ML-Challenge-Team-AutoGen-\student_resource\dataset'

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Sudee\AppData\Local\Temp\ipykernel_22032\299870043.py:2: SyntaxWarning: invalid escape sequence '\P'
  DATASET_PATH = 'D:\Projects\Amazon-ML-Challenge-Team-AutoGen-\student_resource\dataset'


In [6]:
import pandas as pd
import requests
import os
import multiprocessing
import matplotlib.pyplot as plt
from functools import partial

# Function to download a single image
def download_image(url, download_folder):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            image_name = url.split('/')[-1]
            file_path = os.path.join(download_folder, image_name)
            with open(file_path, 'wb') as f:
                f.write(response.content)
        else:
            print(f"Failed to download {url}, status code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {url}: {str(e)}")

# Function to download images in parallel using multiprocessing
def download_images(image_links, download_folder, batch_size=100, max_workers=60):
    os.makedirs(download_folder, exist_ok=True)
    
    # Create a pool of workers with a specified maximum number of processes
    with multiprocessing.Pool(processes=max_workers) as pool:
        # Split image links into batches
        for i in range(0, len(image_links), batch_size):
            batch = image_links[i:i + batch_size]
            # Use pool.map to download images in parallel
            pool.map(partial(download_image, download_folder=download_folder), batch)

# Function to display random images with their entity name and value
def display_random_images(df, num_images=5):
    random_samples = df.sample(num_images)
    plt.figure(figsize=(15, 10))
    
    for i, (index, row) in enumerate(random_samples.iterrows()):
        img_path = os.path.join('dataset/train_images', row['image_link'].split('/')[-1])
        if os.path.exists(img_path):
            img = plt.imread(img_path)
            plt.subplot(1, num_images, i + 1)
            plt.imshow(img)
            plt.title(f"{row['entity_name']}: {row['entity_value']}")
            plt.axis('off')
        else:
            print(f"Image not found: {img_path}")
    
    plt.show()

# Load the training and test datasets
train_df = pd.read_csv('dataset/train.csv')
test_df = pd.read_csv('dataset/test.csv')

# Display total null values in the dataset
print("Total null values in training dataset:")
print(train_df.isnull().sum())

# Replace null values in entity_value with an empty string
train_df['entity_value'] = train_df['entity_value'].fillna('')

# Cell 3: Download Images for both train and test datasets
# Download training images in batches using multiprocessing
print("Downloading training images in parallel...")
download_images(train_df['image_link'].tolist(), download_folder='dataset/train_images', batch_size=100, max_workers=63)

# Download test images in batches using multiprocessing
print("Downloading test images in parallel...")
download_images(test_df['image_link'].tolist(), download_folder='dataset/test_images', batch_size=100, max_workers=63)

# Cell 4: Display 5 random images from the training dataset
display_random_images(train_df, num_images=5)


Total null values in training dataset:
image_link      0
group_id        0
entity_name     0
entity_value    0
dtype: int64


Exception in thread Thread-5 (_handle_workers):
Traceback (most recent call last):
  File "d:\python3.12\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\Sudee\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "d:\python3.12\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "d:\python3.12\Lib\multiprocessing\pool.py", line 522, in _handle_workers
    cls._wait_for_updates(current_sentinels, change_notifier)
  File "d:\python3.12\Lib\multiprocessing\pool.py", line 502, in _wait_for_updates
    wait(sentinels, timeout=timeout)
  File "d:\python3.12\Lib\multiprocessing\connection.py", line 1084, in wait
    ready_handles = _exhaustive_wait(waithandle_to_obj.keys(), timeout)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\python3.12\Lib\multiprocessing\connection.py", line 1016, in _exhaustive_wait
 

In [ ]:
# Cell 5: Group Data by Entity Name
# Group the training data by entity_name
grouped_train_df = train_df.groupby('entity_name')

# Display the grouped data (optional)
for name, group in grouped_train_df:
    print(f"Entity Name: {name}, Number of Samples: {len(group)}")

In [ ]:
# Cell 6: Prepare Data for Model Training
# Create a mapping for allowed units and their abbreviations
unit_abbreviations = {
    # For 'item_weight' and 'maximum_weight_recommendation'
    'gram': ['g', 'gr', 'gm', 'grams', 'grm'],
    'kilogram': ['kg', 'kilograms', 'kgs'],
    'milligram': ['mg', 'milligrams', 'mgs'],
    'microgram': ['µg', 'mcg', 'micrograms'],
    'ounce': ['oz', 'ounces', 'ozs'],
    'pound': ['lb', 'lbs', 'pounds'],
    'ton': ['t', 'tons', 'tonne', 'tonnes'],

    # For 'item_volume'
    'millilitre': ['ml', 'milliliters', 'millilitres'],
    'litre': ['l', 'lit', 'liters', 'litres'],
    'cubic_centimetre': ['cc', 'cm³', 'cubic centimeters', 'cubic centimetres'],
    'cubic_metre': ['m³', 'cubic meters', 'cubic metres'],
    'gallon': ['gal', 'gallons'],
    'quart': ['qt', 'quarts'],
    'pint': ['pt', 'pints'],
    'cup': ['c', 'cups'],

    # For 'voltage'
    'volt': ['v', 'volts'],
    'kilovolt': ['kv', 'kilovolts'],
    'millivolt': ['mv', 'millivolts'],

    # For 'wattage'
    'watt': ['w', 'watts'],
    'kilowatt': ['kw', 'kilowatts'],
    'megawatt': ['mw', 'megawatts'],
    'gigawatt': ['gw', 'gigawatts'],

    # For 'height', 'depth', and 'width'
    'millimetre': ['mm', 'millimeters', 'millimetres'],
    'centimetre': ['cm', 'centimeters', 'centimetres'],
    'metre': ['m', 'meters', 'metres'],
    'kilometre': ['km', 'kilometers', 'kilometres'],
    'inch': ['in', 'inches'],
    'foot': ['ft', 'feet'],
    'yard': ['yd', 'yards'],
    'mile': ['mi', 'miles'],

    # Other common units
    'degree_celsius': ['°C', 'C', 'degrees Celsius'],
    'degree_fahrenheit': ['°F', 'F', 'degrees Fahrenheit'],
    'calorie': ['cal', 'calories'],
    'kilocalorie': ['kcal', 'kcals'],
    'joule': ['j', 'joules'],
    'pascal': ['Pa', 'pascals'],
    'bar': ['bar', 'bars'],
    'psi': ['psi', 'pounds per square inch'],
    'newton': ['N', 'newtons'],
    'fluid_ounce': ['fl oz', 'fluid ounces'],
}

# Prepare the data for BERT encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def create_labels(df):
    labels = []
    for _, row in df.iterrows():
        entity_value = row['entity_value']
        entity_name = row['entity_name']
        
        # Tokenize the entity value
        tokens = tokenizer.tokenize(entity_value)
        
        # Create a label list for the tokens
        label = []
        for token in tokens:
            if token in unit_abbreviations.get(entity_name, []):
                label.append(1)  # Label for the unit (1 for unit)
            else:
                label.append(0)  # Label for non-unit (0 for non-unit)
        
        # Pad the label to match the maximum length
        label += [0] * (tokenizer.model_max_length - len(label))
        
        # Append the label list to the labels
        labels.append(label[:tokenizer.model_max_length])  # Ensure it doesn't exceed max length
    
    return labels

# Generate the training labels
train_labels = create_labels(train_df)

# Prepare the encodings for training
def encode_data(df):
    return tokenizer(df['entity_value'].tolist(), padding=True, truncation=True, return_tensors='pt')

train_encodings = encode_data(train_df)

In [ ]:
# Cell 7: Define the Dataset Class
from torch.utils.data import Dataset

class EntityDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create the dataset
train_dataset = EntityDataset(train_encodings, train_labels)


In [ ]:
# Cell 8: Define the Model
# Load BERT model for token classification
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(entity_unit_map))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
)

# Create a Trainer instance with EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Stop training if no improvement
)


In [ ]:
# Cell 9: Train the Model
# Train the model
trainer.train()

# Save the model after training
trainer.save_model('./saved_model')  # Specify the directory where you want to save the model

In [ ]:
# Cell 10: Evaluate the Model
# Evaluate the model on the training set
train_predictions = trainer.predict(train_dataset)
train_preds = np.argmax(train_predictions.predictions, axis=2)

# Generate classification report
print("Classification Report on Training Set:")
print(classification_report(train_labels, train_preds))

# Calculate and print accuracy
accuracy = accuracy_score(train_labels, train_preds)
print(f"Accuracy on Training Set: {accuracy:.4f}")

In [ ]:
# Cell 11: Make Predictions on Test Set
# Prepare test data for predictions
test_encodings = encode_data(test_df)

# Create test labels (this is a placeholder; you need to define how to create labels)
test_labels = create_labels(test_df)  # Generate test labels similarly

# Create the test dataset
test_dataset = EntityDataset(test_encodings, test_labels)

# Make predictions
test_predictions = trainer.predict(test_dataset)
test_preds = np.argmax(test_predictions.predictions, axis=2)

# Integrate unit extraction into the prediction process
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

def find_units_in_text(text, unit_abbreviations):
    found_units = {}
    for unit, abbreviations in unit_abbreviations.items():
        for abbreviation in abbreviations:
            if abbreviation in text:
                found_units[unit] = abbreviation
                break  # Stop checking once we find a match for this unit
    return found_units

In [ ]:
# Cell 12: Create Submission File
# Create a DataFrame for submission
submission_df = pd.DataFrame({
    'index': test_df['index'],
    'prediction': [entity_unit_map[row['entity_name']][pred] for row, pred in zip(test_df.iterrows(), test_preds)],
    'extracted_units': [find_units_in_text(extract_text_from_image(os.path.join('dataset/test_images', row['image_link'].split('/')[-1])), unit_abbreviations) for _, row in test_df.iterrows()]
})

# Save the DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")


In [ ]:


# Function to process a single image
def process_single_image(index):
    row = test_df.iloc[index]
    img_path = os.path.join('dataset/test_images', row['image_link'].split('/')[-1])
    
    # Extract text from the image
    extracted_text = extract_text_from_image(img_path)
    
    # Find units in the extracted text
    found_units = find_units_in_text(extracted_text, unit_abbreviations)
    
    # Get the prediction for this index
    prediction = [entity_unit_map[row['entity_name']][pred] for pred in test_preds[index]]
    
    # Print the results
    print(f"Image: {row['image_link']}")
    print(f"Extracted Text: {extracted_text}")
    print(f"Found Units: {found_units}")
    print(f"Prediction: {prediction}")

# Example usage for a single image (e.g., the first image in the test set)
process_single_image(0)  # Change the index to process a different image